In [1]:
import os, sys
import django
import pytz
from dateutil import parser
from django.utils.timezone import make_aware
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "esr21.settings"

# Connect to Django ORM
django.setup()
import uuid
from django.contrib.sites.models import Site
from django.core.exceptions import ValidationError


# path = '/Users/imosweu/export_migration_data_esr21/20211115/'
path = '/home/esr21/export_migration_data_esr21/20211115/'

In [ ]:
# Eligibility Confirmation
from esr21_subject.models import EligibilityConfirmation
from esr21_subject.identifiers import ScreeningIdentifier
from django.contrib.sites.models import Site

fname = path + 'eligibility_confirmation.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

sc_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    line.remove('"')
    del line[0]
    
    if len(line) == len(headers):
        data = dict(zip(headers, line))
        
    else:
        raise ValidationError('Line is not the right size')
    sc_data.append(data)


site = Site.objects.get(id=41)
count = 0
already_created = 0

for data in sc_data:
   
    data.update(
        report_datetime=parser.parse(data.get('report_datetime')),
        created=parser.parse(data.get('created')),
        is_consented=data.get('is_consented').strip("\""),
        slug=data.get('slug').strip("\""),
        subject_identifier=data.get('subject_identifier').strip("\""),
        age_in_years=int(data.get('age_in_years'))
    #     id=sc.get('id')
    )
    del data['_state']
    if data['modified']:
        data.update(modified=parser.parse(data.get('modified')))
    else:
        data.update(modified=None)
    eligibility_confirmation = EligibilityConfirmation(**data)
    eligibility_confirmation.save()
    count += 1
print(f'Total created eligibility confirmation {count}')


In [9]:
# Informed Consent
import uuid
from esr21_subject.models import InformedConsent
from django.contrib.sites.models import Site
from django.core.exceptions import ValidationError

fname = path + 'informed_consent.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

s_consent_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    s_consent_data.append(data)
    count += 0
    

count = 0
already_created = 0
pids_dict = {}
    
for dd in s_consent_data:
    del dd['_state']
    created = parser.parse(dd.get('created'))
    consent_datetime = parser.parse(dd.get('consent_datetime'))
    dob = parser.parse(dd.get('dob'))
    dob = dob.date()
    dd.update(
        created=created,
        consent_datetime=consent_datetime,
        report_datetime=parser.parse(dd.get('consent_datetime')),
        dob=dob,
    )
    
    if dd['is_verified_datetime']:
        is_verified_datetime = parser.parse(dd.get('is_verified_datetime'))
        dd.update(
        is_verified_datetime=is_verified_datetime)
    else:
        dd.update(is_verified_datetime=None)
    if dd['modified']:
        modified = parser.parse(dd.get('modified'))
        dd.update(modified=modified)
    else:
        dd.update(modified=None)
    if dd['guardian_name']:
        guardian_name = dd['guardian_name']
        name1, name2 = guardian_name.split(',')
        guardian_name = name1 + ', ' + name2
        dd.update(guardian_name=guardian_name)

    try:
        InformedConsent.objects.get(identity=dd.get('identity'))
    except InformedConsent.DoesNotExist:
        old_id = dd.get('subject_identifier')
        del dd['subject_identifier']
        del dd['id']
        del dd['consent_identifier']
        del dd['slug']
        del dd['subject_identifier_aka']
        del dd['subject_identifier_as_pk']
        
        
        subject_consent = InformedConsent.objects.create(**dd)

        pids_dict[old_id] = subject_consent.subject_identifier
        count += 1
    else:
        already_created += 1
print(f'Total created consents {count}')
print(pids_dict)

{'comment': '',
 'confirm_identity': '103123817',
 'consent_datetime': datetime.datetime(2021, 11, 12, 7, 48, tzinfo=tzutc()),
 'consent_to_participate': 'Yes',
 'created': datetime.datetime(2021, 11, 12, 7, 50, 4, 312473, tzinfo=tzutc()),
 'device_created': '97',
 'device_modified': '97',
 'dm_comment': '',
 'dob': datetime.date(1983, 6, 5),
 'first_name': 'Moya',
 'gender': 'F',
 'gender_other': '',
 'guardian_name': '',
 'hostname_created': 'nodeserver97',
 'hostname_modified': 'nodeserver97',
 'identity': '103123817',
 'identity_type': 'national_identity_card',
 'initials': 'ME',
 'is_dob_estimated': '-',
 'is_incarcerated': '',
 'is_literate': 'Yes',
 'is_verified': 'False',
 'is_verified_datetime': None,
 'language': 'tn',
 'last_name': 'Elija',
 'modified': datetime.datetime(2021, 11, 12, 7, 50, 4, 312478, tzinfo=tzutc()),
 'optional_sample_collection': 'Yes',
 'report_datetime': datetime.datetime(2021, 11, 12, 7, 48, tzinfo=tzutc()),
 'revision': '',
 'screening_identifier': 'S

{'comment': '',
 'confirm_identity': '914414510',
 'consent_datetime': datetime.datetime(2021, 11, 12, 9, 43, tzinfo=tzutc()),
 'consent_to_participate': 'Yes',
 'created': datetime.datetime(2021, 11, 12, 9, 44, 28, 84949, tzinfo=tzutc()),
 'device_created': '97',
 'device_modified': '97',
 'dm_comment': '',
 'dob': datetime.date(1984, 1, 5),
 'first_name': 'MARUNGU',
 'gender': 'M',
 'gender_other': '',
 'guardian_name': '',
 'hostname_created': 'nodeserver97',
 'hostname_modified': 'nodeserver97',
 'identity': '914414510',
 'identity_type': 'national_identity_card',
 'initials': 'MS',
 'is_dob_estimated': '-',
 'is_incarcerated': '',
 'is_literate': 'Yes',
 'is_verified': 'False',
 'is_verified_datetime': None,
 'language': 'tn',
 'last_name': 'SHANGONDE',
 'modified': datetime.datetime(2021, 11, 12, 9, 44, 28, 84956, tzinfo=tzutc()),
 'optional_sample_collection': 'Yes',
 'report_datetime': datetime.datetime(2021, 11, 12, 9, 43, tzinfo=tzutc()),
 'revision': '',
 'screening_identifi

{'comment': '',
 'confirm_identity': '322327012',
 'consent_datetime': datetime.datetime(2021, 11, 12, 11, 25, tzinfo=tzutc()),
 'consent_to_participate': 'Yes',
 'created': datetime.datetime(2021, 11, 12, 11, 26, 28, 280096, tzinfo=tzutc()),
 'device_created': '97',
 'device_modified': '97',
 'dm_comment': '',
 'dob': datetime.date(1992, 12, 2),
 'first_name': 'OBAKENG',
 'gender': 'F',
 'gender_other': '',
 'guardian_name': '',
 'hostname_created': 'nodeserver97',
 'hostname_modified': 'nodeserver97',
 'identity': '322327012',
 'identity_type': 'national_identity_card',
 'initials': 'OM',
 'is_dob_estimated': '-',
 'is_incarcerated': '',
 'is_literate': 'Yes',
 'is_verified': 'False',
 'is_verified_datetime': None,
 'language': 'en',
 'last_name': 'MORWAKWENA',
 'modified': datetime.datetime(2021, 11, 12, 11, 26, 28, 280102, tzinfo=tzutc()),
 'optional_sample_collection': 'Yes',
 'report_datetime': datetime.datetime(2021, 11, 12, 11, 25, tzinfo=tzutc()),
 'revision': '',
 'screening_

{'comment': '',
 'confirm_identity': '780219411',
 'consent_datetime': datetime.datetime(2021, 11, 12, 13, 20, tzinfo=tzutc()),
 'consent_to_participate': 'Yes',
 'created': datetime.datetime(2021, 11, 12, 13, 22, 24, 483203, tzinfo=tzutc()),
 'device_created': '97',
 'device_modified': '97',
 'dm_comment': '',
 'dob': datetime.date(1987, 8, 30),
 'first_name': 'OTHISITSE',
 'gender': 'M',
 'gender_other': '',
 'guardian_name': '',
 'hostname_created': 'nodeserver97',
 'hostname_modified': 'nodeserver97',
 'identity': '780219411',
 'identity_type': 'national_identity_card',
 'initials': 'OJ',
 'is_dob_estimated': '-',
 'is_incarcerated': '',
 'is_literate': 'No',
 'is_verified': 'False',
 'is_verified_datetime': None,
 'language': 'tn',
 'last_name': 'JENAMO',
 'modified': datetime.datetime(2021, 11, 12, 13, 22, 24, 483210, tzinfo=tzutc()),
 'optional_sample_collection': 'Yes',
 'report_datetime': datetime.datetime(2021, 11, 12, 13, 20, tzinfo=tzutc()),
 'revision': '',
 'screening_ide

In [2]:
sids = {'150-041970446-3': '150-041970489-3', '150-041970447-1': '150-041970490-1', 
        '150-041970448-9': '150-041970491-9', '150-041970449-7': '150-041970492-7', 
        '150-041970450-5': '150-041970493-5', '150-041970451-3': '150-041970494-3', 
        '150-041970452-1': '150-041970495-0', '150-041970453-9': '150-041970496-8', 
        '150-041970454-7': '150-041970497-6', '150-041970455-4': '150-041970498-4', 
        '150-041970456-2': '150-041970499-2', '150-041970457-0': '150-041970500-7',
        '150-041970458-8': '150-041970501-5', '150-041970459-6': '150-041970502-3', 
        '150-041970460-4': '150-041970503-1', '150-041970461-2': '150-041970504-9',
        '150-041970462-0': '150-041970505-6', '150-041970463-8': '150-041970506-4',
        '150-041970464-6': '150-041970507-2', '150-041970465-3': '150-041970508-0', 
        '150-041970466-1': '150-041970509-8', '150-041970467-9': '150-041970510-6', 
        '150-041970468-7': '150-041970511-4', '150-041970469-5': '150-041970512-2',
        '150-041970470-3': '150-041970513-0', '150-041970471-1': '150-041970514-8',
        '150-041970472-9': '150-041970515-5', '150-041970473-7': '150-041970516-3',
        '150-041970474-5': '150-041970517-1', '150-041970475-2': '150-041970518-9',
        '150-041970476-0': '150-041970519-7', '150-041970477-8': '150-041970520-5'}

In [3]:
sids.get('150-041970446-3')

'150-041970489-3'

In [9]:
# Screening Eligibility
from esr21_subject.models import ScreeningEligibility
from esr21_subject.identifiers import ScreeningIdentifier
from django.contrib.sites.models import Site

fname = path + 'subject_screening.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

sc_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    
#     import pdb; pdb.set_trace()
    if len(line) == len(headers):
        data = dict(zip(headers, line))
        
    else:
        raise ValidationError('Line is not the right size')
    sc_data.append(data)


site = Site.objects.get(id=41)
count = 0
already_created = 0

for data in sc_data:
   
    data.update(
        report_datetime=parser.parse(data.get('report_datetime')),
        created=parser.parse(data.get('created')),
        slug=data.get('slug').strip("\""),
        subject_identifier=sids.get(data.get('subject_identifier')),
    )
    del data['_state']
    if data['modified']:
        data.update(modified=parser.parse(data.get('modified')))
    else:
        data.update(modified=None)
        
    screening_eligibility = ScreeningEligibility(**data)
    screening_eligibility.save()
    count += 1
print(f'Total created screening eligibility {count}')


--Return--
None
> /var/folders/55/_k910dwn2fvdllz2rvpt7_cw0000gn/T/ipykernel_14044/585302500.py(33)<module>()
     31 count = 0
     32 already_created = 0
---> 33 import pdb; pdb.set_trace()
     34 for data in sc_data:
     35 

ipdb> c
Total created screening eligibility 32


In [8]:
# Personal Contact Information(Still pending)
from esr21_subject.models import PersonalContactInfo
from esr21_subject.identifiers import ScreeningIdentifier
from django.contrib.sites.models import Site

fname = path + 'personal_contact_infos.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

sc_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    
#     import pdb; pdb.set_trace()
    if len(line) == len(headers):
        headers[-1] = headers[-1].strip(',')
        data = dict(zip(headers, line))
        
    else:
        raise ValidationError('Line is not the right size')
    sc_data.append(data)


site = Site.objects.get(id=41)
count = 0
already_created = 0

for data in sc_data:
#     import pdb; pdb.set_trace()
    data.update(
        report_datetime=parser.parse(data.get('report_datetime')),
        created=parser.parse(data.get('created')),
        subject_identifier=sids.get(data.get('subject_identifier')),
    )
    del data['_state']
    
    if data['modified']:
        data.update(modified=parser.parse(data.get('modified')))
    else:
        data.update(modified=None)
    #     try:
    #         EligibilityConfirmation.objects.get(id=sc.get('id'))
    #     except EligibilityConfirmation.DoesNotExist:
    #         sc.update(screening_identifier=ScreeningIdentifier().identifier)
    personal_contact_info = PersonalContactInfo(**data)
    personal_contact_info.save_base(raw=True)
    count += 1
    #     else:
    #         already_created += 1
print(f'Total created personal contact info {count}')


Total created personal contact info 31


In [6]:
# Import appointments
from edc_appointment.models import Appointment

fname = path + 'appointments.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


ap_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    ap = dict(zip(headers, line))
    ap_data.append(ap)
    
site = Site.objects.get(id=41)
count = 0
already_created = 0
for ap in ap_data:
    ap.update(
        created=parser.parse(ap.get('created')),
        facility_name='5-day clinic',
    )


    if ap['modified']:
        ap.update(modified=parser.parse(ap.get('modified')))
    else:
        ap.update(modified=None)

    if ap['appt_datetime']:
        ap.update(
        appt_datetime=parser.parse(ap.get('appt_datetime')))
    else:
        ap.update(appt_datetime=None)

    if ap['timepoint_datetime']:
        ap.update(
        timepoint_datetime=parser.parse(ap.get('timepoint_datetime')))
    else:
        ap.update(timepoint_datetime=None)

    if ap['appt_close_datetime']:
        ap.update(
        appt_close_datetime=parser.parse(ap.get('appt_close_datetime')))
    else:
        ap.update(appt_close_datetime=None)

    ap.update(timepoint_closed_datetime=None)

    del ap['_state']

    appt = Appointment.objects.get(subject_identifier=sids.get(ap.get('subject_identifier')), visit_code='1000')
    appt.appt_datetime = ap.get('appt_datetime')
    appt.appt_type = ap.get('appt_type')
    appt.appt_status = ap.get('appt_status')
    appt.appt_reason = ap.get('appt_reason')
    appt.is_confirmed = ap.get('is_confirmed')
    appt.save_base(raw=True)
    count += 1


print(f'Total created subject appointment {count}')


Total created subject appointment 0


In [19]:
# Subject visit
from datetime import timedelta
from esr21_subject.models import SubjectVisit
from edc_appointment.models import Appointment
from django.db.utils import IntegrityError

fname = path + 'subject_visits.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


sv_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    sv_data.append(data)

count = 0
already_created = 0
for sv in sv_data:
    del sv['_state']
    sv.update(
        created=parser.parse(sv.get('created')),
    )
    if sv['report_datetime']:
        sv.update(
            report_datetime=parser.parse(sv.get('report_datetime')))
    else:
        sv.update(report_datetime=None)
    if sv['modified']:
        sv.update(
            modified=parser.parse(sv.get('modified')))
    else:
        sv.update(modified=None)
    if sv['last_alive_date']:
        sv.update(
            last_alive_date=parser.parse(sv.get('last_alive_date'))).date()
    else:
        sv.update(last_alive_date=None)
    
    try:
        appointment = Appointment.objects.get(subject_identifier=sids.get(sv.get('subject_identifier')),
                                             visit_code=sv.get('visit_code'))
    except Appointment.DoesNotExist:
        pass
    else:
        sv.update(
                appointment_id=appointment.id,
                appointment=appointment,)
    
    try:
        sub_v = SubjectVisit.objects.get(id=sv.get('id'))
    except SubjectVisit.DoesNotExist:
        maternal_visit = SubjectVisit(**sv)
        maternal_visit.save()
        count += 1

print(f'Total created subject visits {count}')


Total created subject visits 0
Total created subject visits 32


In [10]:
count

0

In [20]:
# Demographics Data
from datetime import timedelta
from esr21_subject.models import SubjectVisit
from esr21_subject.models import DemographicsData
from django.db.utils import IntegrityError

fname = path + 'subject_crfs/demographicsdata.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


dm_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    dm_data.append(data)

count = 0
already_created = 0
for dm in dm_data:
    del dm['_state']
    dm.update(
        created=parser.parse(dm.get('created')),
    )
    if dm['report_datetime']:
        dm.update(
            report_datetime=parser.parse(dm.get('report_datetime')))
    else:
        dm.update(report_datetime=None)
    if dm['modified']:
        dm.update(
            modified=parser.parse(dm.get('modified')))
    else:
        dm.update(modified=None)
    
    crf = DemographicsData(**dm)
    crf.save()
    count += 1
print(f'Total created demographic data {count}')


Total created demographic data 32


In [22]:
# Medical History
from datetime import timedelta
from esr21_subject.models import SubjectVisit
from esr21_subject.models import MedicalHistory
from django.db.utils import IntegrityError

fname = path + 'subject_crfs/medicalhistory.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


dm_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    dm_data.append(data)

count = 0
already_created = 0
for dm in dm_data:
#     import pdb; pdb.set_trace()
    dm.update(
        created=parser.parse(dm.get('created')),
    )
    if dm['report_datetime']:
        dm.update(
            report_datetime=parser.parse(dm.get('report_datetime')))
    else:
        dm.update(report_datetime=None)
    if dm['modified']:
        dm.update(
            modified=parser.parse(dm.get('modified')))
    else:
        dm.update(modified=None)
    
    del dm['_state']
    try:
        SubjectVisit.objects.get(id=dm.get('subject_visit_id'))
    except SubjectVisit.DoesNotExist:
        pass
    else:
        crf = MedicalHistory(**dm)
        crf.save()
        count += 1
  
print(f'Total created medical history {count}')


Total created medical history 32


In [25]:
# Physical Exam
from datetime import timedelta
from esr21_subject.models import SubjectVisit
from esr21_subject.models import PhysicalExam
from django.db.utils import IntegrityError

fname = path + 'subject_crfs/physicalexam.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


dm_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    dm_data.append(data)

count = 0
already_created = 0
for dm in dm_data:

    dm.update(
        created=parser.parse(dm.get('created')),
    )
    if dm['report_datetime']:
        dm.update(
            report_datetime=parser.parse(dm.get('report_datetime')))
    else:
        dm.update(report_datetime=None)
    if dm['modified']:
        dm.update(
            modified=parser.parse(dm.get('modified')))
    else:
        dm.update(modified=None)
    
    del dm['_state']
    try:
        sv = SubjectVisit.objects.get(id=dm.get('subject_visit_id'))
    except SubjectVisit.DoesNotExist:
        pass
    else:
        dm.update(
            exam_date=parser.parse(dm.get('exam_date')))

        crf = PhysicalExam(**dm)
        crf.save()
        count += 1
print(f'Total created physical exam {count}')


Total created physical exam 30


In [30]:
#RapidHIVTesting
from datetime import timedelta
from esr21_subject.models import SubjectVisit
from esr21_subject.models import RapidHIVTesting
from django.db.utils import IntegrityError

fname = path + 'subject_crfs/rapidhivtesting.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


dm_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    dm_data.append(data)

count = 0
already_created = 0
for dm in dm_data:

    del dm['_state']
    dm.update(
        created=parser.parse(dm.get('created')),
    )
    if dm['report_datetime']:
        dm.update(
            report_datetime=parser.parse(dm.get('report_datetime')))
    else:
        dm.update(report_datetime=None)
    if dm['modified']:
        dm.update(
            modified=parser.parse(dm.get('modified')))
    else:
        dm.update(modified=None)
    if dm['rapid_test_date']:
        dm.update(
            rapid_test_date=parser.parse(dm.get('rapid_test_date')))
    else:
        dm.update(rapid_test_date=None)
        
    if dm['hiv_test_date']:
        dm.update(
            hiv_test_date=parser.parse(dm.get('hiv_test_date')))
    else:
        dm.update(hiv_test_date=None)
    
    try:
        sv = SubjectVisit.objects.get(id=dm.get('subject_visit_id'))
    except SubjectVisit.DoesNotExist:
        pass
    else:
        crf = RapidHIVTesting(**dm)
        crf.save()
        count += 1
print(f'Total created rapid hiv test {count}')


Total created rapid hiv test 32


In [32]:
#vitalsigns
from datetime import timedelta
from esr21_subject.models import SubjectVisit
from esr21_subject.models import VitalSigns
from django.db.utils import IntegrityError

fname = path + 'subject_crfs/vitalsigns.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


dm_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    dm_data.append(data)

count = 0
already_created = 0
for dm in dm_data:

    del dm['_state']
    dm.update(
        created=parser.parse(dm.get('created')),
    )
    if dm['report_datetime']:
        dm.update(
            report_datetime=parser.parse(dm.get('report_datetime')))
    else:
        dm.update(report_datetime=None)
    if dm['modified']:
        dm.update(
            modified=parser.parse(dm.get('modified')))
    else:
        dm.update(modified=None)
    
    try:
        sv = SubjectVisit.objects.get(id=dm.get('subject_visit_id'))
    except SubjectVisit.DoesNotExist:
        pass
    else:
        dm.update(
                assessment_dt=parser.parse(dm.get('assessment_dt'))
        )

        crf = VitalSigns(**dm)
        crf.save()
        count += 1
print(f'Total created vitalsigns {count}')

Total created vitalsigns 32


In [33]:
#Covid19PreventativeBehaviours
from datetime import timedelta
from esr21_subject.models import SubjectVisit
from esr21_subject.models import Covid19PreventativeBehaviours
from django.db.utils import IntegrityError

fname = path + 'subject_crfs/covid19preventativebehaviours.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


dm_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    dm_data.append(data)

count = 0
already_created = 0
for dm in dm_data:
    
    del dm['_state']
    
    dm.update(
        created=parser.parse(dm.get('created')),
    )
    if dm['report_datetime']:
        dm.update(
            report_datetime=parser.parse(dm.get('report_datetime')))
    else:
        dm.update(report_datetime=None)
    if dm['modified']:
        dm.update(
            modified=parser.parse(dm.get('modified')))
    else:
        dm.update(modified=None)
    
    try:
        sv = SubjectVisit.objects.get(id=dm.get('subject_visit_id'))
    except SubjectVisit.DoesNotExist:
        pass
    else:
        crf = Covid19PreventativeBehaviours(**dm)
        crf.save()
        count += 1
print(f'Total created Covid19PreventativeBehaviours {count}')


Total created Covid19PreventativeBehaviours 32


In [35]:
#VaccinationDetails
from datetime import timedelta
from esr21_subject.models import SubjectVisit
from esr21_subject.models import VaccinationDetails
from django.db.utils import IntegrityError

fname = path + 'subject_crfs/vaccinationdetails.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


dm_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    dm_data.append(data)

count = 0
already_created = 0
for dm in dm_data:
    del dm['_state']
    
    dm.update(
        created=parser.parse(dm.get('created')),
    )
    if dm['report_datetime']:
        dm.update(
            report_datetime=parser.parse(dm.get('report_datetime')))
    else:
        dm.update(report_datetime=None)
    if dm['modified']:
        dm.update(
            modified=parser.parse(dm.get('modified')))
    else:
        dm.update(modified=None)
    
    try:
        sv = SubjectVisit.objects.get(id=dm.get('subject_visit_id'))
    except SubjectVisit.DoesNotExist:
        pass
    else:
        dm.update(
                expiry_date=parser.parse(dm.get('expiry_date')),
                vaccination_date=parser.parse(dm.get('vaccination_date')).date(),
            next_vaccination_date=parser.parse(dm.get('next_vaccination_date')).date()
            
        )
        crf = VaccinationDetails(**dm)
        crf.save()
        count += 1
print(f'Total created VaccinationDetails {count}')

/Users/imosweu/.venvs/esr21-live/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1309: RuntimeWarning: DateTimeField VaccinationDetails.vaccination_date received a naive datetime (2021-11-12 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "


Total created VaccinationDetails 30


In [4]:
#Pregnancy Status
from datetime import timedelta
from esr21_subject.models import SubjectVisit
from esr21_subject.models import PregnancyStatus
from django.db.utils import IntegrityError

fname = path + 'subject_crfs/pregnancystatus.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


dm_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    dm_data.append(data)

count = 0
already_created = 0
for dm in dm_data:
    
    del dm['_state']
    
    dm.update(
        created=parser.parse(dm.get('created')),
    )
    if dm['report_datetime']:
        dm.update(
            report_datetime=parser.parse(dm.get('report_datetime')))
    else:
        dm.update(report_datetime=None)
    if dm['modified']:
        dm.update(
            modified=parser.parse(dm.get('modified')))
    else:
        dm.update(modified=None)
        
    if dm['start_date_menstrual_period']:
        dm.update(
            start_date_menstrual_period=parser.parse(dm.get('start_date_menstrual_period')))
    else:
        dm.update(start_date_menstrual_period=None)
        
    if dm['expected_delivery']:
        dm.update(
            expected_delivery=parser.parse(dm.get('expected_delivery')))
    else:
        dm.update(expected_delivery=None)
        
    if dm['number_prev_pregnancies']:
        dm.update(
            number_prev_pregnancies=int(float(dm.get('number_prev_pregnancies'))))
    else:
        dm.update(number_prev_pregnancies=None)
        
    if dm['number_normal_pregnancies']:
        dm.update(
            number_normal_pregnancies=int(float(dm.get('number_normal_pregnancies'))))
    else:
        dm.update(number_normal_pregnancies=None)
        
    if dm['number_miscarriages']:
        dm.update(
            number_miscarriages=int(float(dm.get('number_miscarriages'))))
    else:
        dm.update(number_miscarriages=None)
        
    if dm['date_miscarriages']:
        dm.update(
            date_miscarriages=parser.parse(dm.get('date_miscarriages')))
    else:
        dm.update(date_miscarriages=None)
    
    
    try:
        sv = SubjectVisit.objects.get(id=dm.get('subject_visit_id'))
    except SubjectVisit.DoesNotExist:
        pass
    else:
        crf = PregnancyStatus(**dm)
        crf.save()
        count += 1
print(f'Total created PregnancyStatus {count}')


Total created PregnancyStatus 16


In [6]:
#Pregnancy test
from datetime import timedelta
from esr21_subject.models import SubjectVisit
from esr21_subject.models import PregnancyTest
from django.db.utils import IntegrityError

fname = path + 'subject_crfs/pregnancytest.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


dm_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    dm_data.append(data)

count = 0
already_created = 0
for dm in dm_data:
    
    del dm['_state']
    
    dm.update(
        created=parser.parse(dm.get('created')),
    )
    if dm['report_datetime']:
        dm.update(
            report_datetime=parser.parse(dm.get('report_datetime')))
    else:
        dm.update(report_datetime=None)
    if dm['modified']:
        dm.update(
            modified=parser.parse(dm.get('modified')))
    else:
        dm.update(modified=None)
        
    if dm['preg_date']:
        dm.update(
            preg_date=parser.parse(dm.get('preg_date')))
    else:
        dm.update(preg_date=None)
    
    try:
        sv = SubjectVisit.objects.get(id=dm.get('subject_visit_id'))
    except SubjectVisit.DoesNotExist:
        pass
    else:
        crf = PregnancyTest(**dm)
        crf.save()
        count += 1
print(f'Total created Covid19PreventativeBehaviours {count}')


Total created Covid19PreventativeBehaviours 15


In [ ]:
# Import appointments
from edc_appointment.models import Appointment

fname = path + 'appointments.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


ap_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    ap = dict(zip(headers, line))
    ap_data.append(ap)
    
site = Site.objects.get(id=41)
count = 0
already_created = 0
for ap in ap_data:
    ap.update(
        created=parser.parse(ap.get('created')),
        facility_name='5-day clinic',
    )


    if ap['modified']:
        ap.update(modified=parser.parse(ap.get('modified')))
    else:
        ap.update(modified=None)

    if ap['appt_datetime']:
        ap.update(
        appt_datetime=parser.parse(ap.get('appt_datetime')))
    else:
        ap.update(appt_datetime=None)

    if ap['timepoint_datetime']:
        ap.update(
        timepoint_datetime=parser.parse(ap.get('timepoint_datetime')))
    else:
        ap.update(timepoint_datetime=None)

    if ap['appt_close_datetime']:
        ap.update(
        appt_close_datetime=parser.parse(ap.get('appt_close_datetime')))
    else:
        ap.update(appt_close_datetime=None)

    ap.update(timepoint_closed_datetime=None)

    del ap['_state']

    appt = Appointment.objects.get(subject_identifier=sids.get(ap.get('subject_identifier')), visit_code='1000')
    appt.appt_datetime = ap.get('appt_datetime')
    appt.appt_type = ap.get('appt_type')
    appt.appt_status = ap.get('appt_status')
    appt.appt_reason = ap.get('appt_reason')
    appt.is_confirmed = ap.get('is_confirmed')
    appt.save_base(raw=True)
    count += 1


print(f'Total created subject appointment {count}')